In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Dense, TimeDistributed, LSTM, InputLayer, 
    Flatten, GRU, Bidirectional, Activation, Reshape, concatenate)
from tensorflow import keras


In [2]:
# surrogate data
n_channels = 32
n_timepoints = 40
n_samples = 100
n_dipoles = 1234

X = np.random.randn(n_samples, n_timepoints, n_channels)
y = x = np.random.randn(n_samples, n_timepoints, n_dipoles)

In [24]:
# NEW MODEL
n_dense_units = 100
n_lstm_units = 25
activation_function = 'relu'
dropout = 0.2

inputs = keras.Input(shape=(None, n_channels), name='Input')
# SINGLE TIME FRAME PATH
fc1 = TimeDistributed(Dense(n_dense_units), name='FC1')(inputs)
fc2 = TimeDistributed(Dense(n_dipoles), name='FC2')(fc1)

model_s = keras.Model(inputs=inputs, outputs=fc2, name='single_time_ frame_model')
# model_s.summary()

# # MULTI TIME FRAME PATH
lstm1 = Bidirectional(LSTM(n_lstm_units, return_sequences=True, 
    input_shape=(None, n_dense_units), dropout=dropout, 
    activation=activation_function), name='LSTM1')(fc1)

concat = concatenate([lstm1, fc2], name='Concat')

lstm2 = Bidirectional(LSTM(n_lstm_units, return_sequences=True, 
    input_shape=(None, n_dense_units), dropout=dropout, 
    activation=activation_function), name='LSTM2')(concat)

output = TimeDistributed(Dense(n_dipoles), name='FC_Out')(lstm2)
model_m = keras.Model(inputs=inputs, outputs=output, name='multi_time_frame_model')

model_m.summary()
model_m.compile(loss='mse')
model_m.fit(X, y)

assert model_m.predict(X).shape == y.shape, 'shape mismatch'

Model: "multi_time_frame_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, None, 32)]   0                                            
__________________________________________________________________________________________________
FC1 (TimeDistributed)           (None, None, 100)    3300        Input[0][0]                      
__________________________________________________________________________________________________
LSTM1 (Bidirectional)           (None, None, 50)     25200       FC1[0][0]                        
__________________________________________________________________________________________________
FC2 (TimeDistributed)           (None, None, 1234)   124634      FC1[0][0]                        
_____________________________________________________________________________

In [26]:
model_m.predict(np.random.randn(1, 11, 32)).

(1, 11, 1234)

In [ ]:
# OLD MODEL
model = tf.keras.Sequential()
tf.keras.backend.set_image_data_format('channels_last')
input_shape = (None, n_channels)
model.add(InputLayer(input_shape=input_shape))

# LSTM layers
for _ in range(1):
    model.add(Bidirectional(LSTM(20, 
        return_sequences=True, input_shape=input_shape, 
        dropout=0.2, activation='relu')))

# Hidden Dense layer(s):
for _ in range(1):
    model.add(Dense(100, 
        activation='relu'))

# Final For-each layer:
model.add(TimeDistributed(
    Dense(n_dipoles, activation='linear'))
)

model.build(input_shape=input_shape)

assert model.predict(X).shape == y.shape, 'shape mismatch'